In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

pd.options.mode.chained_assignment = None

In [2]:
trades = pd.read_csv('trades.csv')
eod_positions = pd.read_csv('eod_positions.csv')
price_updates = pd.read_csv('price_updates.csv')

In [3]:
trades['time'] = pd.to_datetime(trades['time'])
price_updates['time'] = pd.to_datetime(price_updates['time'])
price_updates.rename(columns = {'sym':'symbol'}, inplace=True)
price_updates['time'] = pd.to_datetime(price_updates['time'])
price_updates.sort_values(by='time', inplace=True)

In [4]:
sod_positions = eod_positions
sod_positions['time'] = pd.to_datetime('09:30')
sod_positions['trade_size'] = sod_positions['position']
column_order = ['time', 'symbol','position','eod_price', 'trade_size']
sod_positions = sod_positions[column_order]
sod_positions.loc[:,'trade_type'] = ['SOD']*sod_positions.shape[0]
sod_positions

,time,symbol,position,eod_price,trade_size,trade_type
0,2024-11-17 09:30:00,MSFT,5000,420,5000,SOD
1,2024-11-17 09:30:00,GOOG,-2000,178,-2000,SOD
2,2024-11-17 09:30:00,META,3000,579,3000,SOD
3,2024-11-17 09:30:00,AMZN,10,189,10,SOD
4,2024-11-17 09:30:00,NFLX,100,749,100,SOD
5,2024-11-17 09:30:00,AAPL,1200,232,1200,SOD


In [5]:
trades['time'] = pd.to_datetime(trades['time'])
trades.loc[:,'trade_type'] = ['ID']*trades.shape[0]
trades

,time,symbol,trade_size,trade_price,trade_type
0,2024-11-17 10:10:00,MSFT,1000,420,ID
1,2024-11-17 10:30:00,GOOG,-1000,172,ID
2,2024-11-17 10:33:00,AAPL,200,235,ID
3,2024-11-17 10:37:00,GOOG,-1000,171,ID
4,2024-11-17 11:15:00,AMZN,200,190,ID
5,2024-11-17 12:00:00,AMZN,300,185,ID
6,2024-11-17 12:15:00,NFLX,20,751,ID
7,2024-11-17 12:20:00,NFLX,20,752,ID
8,2024-11-17 12:30:00,NFLX,-10,753,ID
9,2024-11-17 12:34:00,AAPL,200,234,ID


In [6]:
total_positions = pd.concat([sod_positions, trades]).reset_index(drop=True)
trades_with_price = pd.merge_asof(left=total_positions, right=price_updates, on='time', by='symbol', allow_exact_matches=True, direction='backward')


In [7]:
#trade price for sod trades is mid price
trades_with_price.loc[trades_with_price['trade_type']=='SOD', 'trade_price'] = trades_with_price.loc[trades_with_price['trade_type']=='SOD', 'mid_price']
trades_with_price


,time,symbol,position,eod_price,trade_size,trade_type,trade_price,mid_price
0,2024-11-17 09:30:00,MSFT,5000.0,420.0,5000,SOD,421.0,421
1,2024-11-17 09:30:00,GOOG,-2000.0,178.0,-2000,SOD,170.0,170
2,2024-11-17 09:30:00,META,3000.0,579.0,3000,SOD,579.0,579
3,2024-11-17 09:30:00,AMZN,10.0,189.0,10,SOD,189.0,189
4,2024-11-17 09:30:00,NFLX,100.0,749.0,100,SOD,750.0,750
5,2024-11-17 09:30:00,AAPL,1200.0,232.0,1200,SOD,233.0,233
6,2024-11-17 10:10:00,MSFT,NaN,NaN,1000,ID,420.0,424
7,2024-11-17 10:30:00,GOOG,NaN,NaN,-1000,ID,172.0,170
8,2024-11-17 10:33:00,AAPL,NaN,NaN,200,ID,235.0,233
9,2024-11-17 10:37:00,GOOG,NaN,NaN,-1000,ID,171.0,167


In [8]:
trades_with_price['cumulative_position'] = trades_with_price.loc[:,['symbol', 'trade_size']].groupby('symbol').cumsum()
trades_with_price

,time,symbol,position,eod_price,trade_size,trade_type,trade_price,mid_price,cumulative_position
0,2024-11-17 09:30:00,MSFT,5000.0,420.0,5000,SOD,421.0,421,5000
1,2024-11-17 09:30:00,GOOG,-2000.0,178.0,-2000,SOD,170.0,170,-2000
2,2024-11-17 09:30:00,META,3000.0,579.0,3000,SOD,579.0,579,3000
3,2024-11-17 09:30:00,AMZN,10.0,189.0,10,SOD,189.0,189,10
4,2024-11-17 09:30:00,NFLX,100.0,749.0,100,SOD,750.0,750,100
5,2024-11-17 09:30:00,AAPL,1200.0,232.0,1200,SOD,233.0,233,1200
6,2024-11-17 10:10:00,MSFT,NaN,NaN,1000,ID,420.0,424,6000
7,2024-11-17 10:30:00,GOOG,NaN,NaN,-1000,ID,172.0,170,-3000
8,2024-11-17 10:33:00,AAPL,NaN,NaN,200,ID,235.0,233,1400
9,2024-11-17 10:37:00,GOOG,NaN,NaN,-1000,ID,171.0,167,-4000


In [9]:
#overnight pnl
sod_filt = trades_with_price['trade_type'] == 'SOD'
trades_with_price.loc[sod_filt, 'overnight_pnl'] = trades_with_price.loc[sod_filt, 'position']*(trades_with_price.loc[sod_filt, 'trade_price'] - trades_with_price.loc[sod_filt, 'eod_price'])
trades_with_price




,time,symbol,position,eod_price,trade_size,trade_type,trade_price,mid_price,cumulative_position,overnight_pnl
0,2024-11-17 09:30:00,MSFT,5000.0,420.0,5000,SOD,421.0,421,5000,5000.0
1,2024-11-17 09:30:00,GOOG,-2000.0,178.0,-2000,SOD,170.0,170,-2000,16000.0
2,2024-11-17 09:30:00,META,3000.0,579.0,3000,SOD,579.0,579,3000,0.0
3,2024-11-17 09:30:00,AMZN,10.0,189.0,10,SOD,189.0,189,10,0.0
4,2024-11-17 09:30:00,NFLX,100.0,749.0,100,SOD,750.0,750,100,100.0
5,2024-11-17 09:30:00,AAPL,1200.0,232.0,1200,SOD,233.0,233,1200,1200.0
6,2024-11-17 10:10:00,MSFT,NaN,NaN,1000,ID,420.0,424,6000,NaN
7,2024-11-17 10:30:00,GOOG,NaN,NaN,-1000,ID,172.0,170,-3000,NaN
8,2024-11-17 10:33:00,AAPL,NaN,NaN,200,ID,235.0,233,1400,NaN
9,2024-11-17 10:37:00,GOOG,NaN,NaN,-1000,ID,171.0,167,-4000,NaN


In [10]:
#trading pnl
id_filt = trades_with_price['trade_type'] == 'ID'
trades_with_price.loc[id_filt, 'trading_pnl'] = trades_with_price.loc[id_filt, 'trade_size']*(trades_with_price.loc[id_filt, 'mid_price'] - trades_with_price.loc[id_filt, 'trade_price'])
trades_with_price


,time,symbol,position,eod_price,trade_size,trade_type,trade_price,mid_price,cumulative_position,overnight_pnl,trading_pnl
0,2024-11-17 09:30:00,MSFT,5000.0,420.0,5000,SOD,421.0,421,5000,5000.0,NaN
1,2024-11-17 09:30:00,GOOG,-2000.0,178.0,-2000,SOD,170.0,170,-2000,16000.0,NaN
2,2024-11-17 09:30:00,META,3000.0,579.0,3000,SOD,579.0,579,3000,0.0,NaN
3,2024-11-17 09:30:00,AMZN,10.0,189.0,10,SOD,189.0,189,10,0.0,NaN
4,2024-11-17 09:30:00,NFLX,100.0,749.0,100,SOD,750.0,750,100,100.0,NaN
5,2024-11-17 09:30:00,AAPL,1200.0,232.0,1200,SOD,233.0,233,1200,1200.0,NaN
6,2024-11-17 10:10:00,MSFT,NaN,NaN,1000,ID,420.0,424,6000,NaN,4000.0
7,2024-11-17 10:30:00,GOOG,NaN,NaN,-1000,ID,172.0,170,-3000,NaN,2000.0
8,2024-11-17 10:33:00,AAPL,NaN,NaN,200,ID,235.0,233,1400,NaN,-400.0
9,2024-11-17 10:37:00,GOOG,NaN,NaN,-1000,ID,171.0,167,-4000,NaN,4000.0


In [11]:
#adding 0 to nan values
trades_with_price['overnight_pnl'] = trades_with_price['overnight_pnl'].fillna(0)
trades_with_price['trading_pnl'] = trades_with_price['trading_pnl'].fillna(0)
trades_with_price.head(30)

,time,symbol,position,eod_price,trade_size,trade_type,trade_price,mid_price,cumulative_position,overnight_pnl,trading_pnl
0,2024-11-17 09:30:00,MSFT,5000.0,420.0,5000,SOD,421.0,421,5000,5000.0,0.0
1,2024-11-17 09:30:00,GOOG,-2000.0,178.0,-2000,SOD,170.0,170,-2000,16000.0,0.0
2,2024-11-17 09:30:00,META,3000.0,579.0,3000,SOD,579.0,579,3000,0.0,0.0
3,2024-11-17 09:30:00,AMZN,10.0,189.0,10,SOD,189.0,189,10,0.0,0.0
4,2024-11-17 09:30:00,NFLX,100.0,749.0,100,SOD,750.0,750,100,100.0,0.0
5,2024-11-17 09:30:00,AAPL,1200.0,232.0,1200,SOD,233.0,233,1200,1200.0,0.0
6,2024-11-17 10:10:00,MSFT,NaN,NaN,1000,ID,420.0,424,6000,0.0,4000.0
7,2024-11-17 10:30:00,GOOG,NaN,NaN,-1000,ID,172.0,170,-3000,0.0,2000.0
8,2024-11-17 10:33:00,AAPL,NaN,NaN,200,ID,235.0,233,1400,0.0,-400.0
9,2024-11-17 10:37:00,GOOG,NaN,NaN,-1000,ID,171.0,167,-4000,0.0,4000.0


In [12]:
trades_with_price['overnight_pnl'] = trades_with_price.loc[:,['symbol','overnight_pnl']].groupby('symbol').cumsum()
trades_with_price['trading_pnl'] = trades_with_price.loc[:,['symbol','trading_pnl']].groupby('symbol').cumsum()
trades_with_price.head(30)

,time,symbol,position,eod_price,trade_size,trade_type,trade_price,mid_price,cumulative_position,overnight_pnl,trading_pnl
0,2024-11-17 09:30:00,MSFT,5000.0,420.0,5000,SOD,421.0,421,5000,5000.0,0.0
1,2024-11-17 09:30:00,GOOG,-2000.0,178.0,-2000,SOD,170.0,170,-2000,16000.0,0.0
2,2024-11-17 09:30:00,META,3000.0,579.0,3000,SOD,579.0,579,3000,0.0,0.0
3,2024-11-17 09:30:00,AMZN,10.0,189.0,10,SOD,189.0,189,10,0.0,0.0
4,2024-11-17 09:30:00,NFLX,100.0,749.0,100,SOD,750.0,750,100,100.0,0.0
5,2024-11-17 09:30:00,AAPL,1200.0,232.0,1200,SOD,233.0,233,1200,1200.0,0.0
6,2024-11-17 10:10:00,MSFT,NaN,NaN,1000,ID,420.0,424,6000,5000.0,4000.0
7,2024-11-17 10:30:00,GOOG,NaN,NaN,-1000,ID,172.0,170,-3000,16000.0,2000.0
8,2024-11-17 10:33:00,AAPL,NaN,NaN,200,ID,235.0,233,1400,1200.0,-400.0
9,2024-11-17 10:37:00,GOOG,NaN,NaN,-1000,ID,171.0,167,-4000,16000.0,6000.0


In [13]:
pd.merge_asof(price_updates, trades_with_price.drop(['position', 'eod_price','trade_size', 'trade_type', 'trade_price', 'mid_price'], axis=1), on='time', by='symbol', allow_exact_matches=True, direction='backward')



,time,symbol,mid_price,cumulative_position,overnight_pnl,trading_pnl
0,2024-11-17 09:00:00,AMZN,189,NaN,NaN,NaN
1,2024-11-17 09:00:00,GOOG,170,NaN,NaN,NaN
2,2024-11-17 09:00:00,NFLX,750,NaN,NaN,NaN
3,2024-11-17 09:00:00,META,579,NaN,NaN,NaN
4,2024-11-17 09:00:00,AAPL,233,NaN,NaN,NaN
...,...,...,...,...,...,...
403,2024-11-17 15:30:00,NFLX,751,1340.0,100.0,-350.0
404,2024-11-17 15:30:00,META,583,3400.0,0.0,6500.0
405,2024-11-17 15:30:00,AMZN,190,510.0,0.0,600.0
406,2024-11-17 15:30:00,MSFT,423,9000.0,5000.0,-11000.0


In [14]:
#mtm pnl
all_pnl = pd.merge_asof(price_updates, trades_with_price.drop(['position', 'eod_price','trade_size', 'trade_type', 'trade_price', 'mid_price'], axis=1), on='time', by='symbol', allow_exact_matches=True, direction='backward')
all_pnl



,time,symbol,mid_price,cumulative_position,overnight_pnl,trading_pnl
0,2024-11-17 09:00:00,AMZN,189,NaN,NaN,NaN
1,2024-11-17 09:00:00,GOOG,170,NaN,NaN,NaN
2,2024-11-17 09:00:00,NFLX,750,NaN,NaN,NaN
3,2024-11-17 09:00:00,META,579,NaN,NaN,NaN
4,2024-11-17 09:00:00,AAPL,233,NaN,NaN,NaN
...,...,...,...,...,...,...
403,2024-11-17 15:30:00,NFLX,751,1340.0,100.0,-350.0
404,2024-11-17 15:30:00,META,583,3400.0,0.0,6500.0
405,2024-11-17 15:30:00,AMZN,190,510.0,0.0,600.0
406,2024-11-17 15:30:00,MSFT,423,9000.0,5000.0,-11000.0


In [15]:
#calculate mtm pnl
all_pnl['delta_mid_price'] = all_pnl.loc[:,['symbol','mid_price']].groupby('symbol').diff()
all_pnl['mtm_pnl'] = all_pnl['delta_mid_price'] * all_pnl['cumulative_position']
all_pnl['mtm_pnl'] = all_pnl['mtm_pnl'].fillna(0)

In [16]:
all_pnl['total_pnl'] = all_pnl['overnight_pnl'] + all_pnl['trading_pnl'] + all_pnl['mtm_pnl']
mtm_pnl.tail


NameError: name 'mtm_pnl' is not defined

In [ ]:
all_pnl.loc[:, ['time','overnight_pnl','trading_pnl', 'total_pnl']]

In [ ]:
book_pnl = all_pnl.groupby(['time']).sum()
book_pnl['symbol'] = 'book'
book_pnl.reset_index(inplace=True)

In [ ]:
book_pnl = book_pnl.loc[:,['time', 'symbol', 'overnight_pnl', 'trading_pnl', 'mtm_pnl', 'total_pnl']]
book_pnl


In [ ]:
sns.lineplot(data = book_pnl, x='time', y='total_pnl')
sns.lineplot(data = book_pnl, x='time', y='mtm_pnl')
sns.lineplot(data = book_pnl, x='time', y='trading_pnl')
sns.lineplot(data = book_pnl, x='time', y='overnight_pnl')



In [ ]:
book_pnl_plot = book_pnl.drop('symbol', axis=1).melt('time', var_name='pnl_type', value_name = 'pnl')

In [ ]:
sns.lineplot(data = book_pnl_plot, x='time', y = 'pnl', hue = 'pnl_type')